# 第四讲 数据整合和数据清洗
- pandas学习参考： [十分钟搞定pandas](http://www.cnblogs.com/chaosimple/p/4153083.html)

## 4.1 SQL语句介绍

In [ ]:
#import os

#os.chdir('E:/data')

In [ ]:
import pandas as pd
import numpy as np

sale = pd.read_csv('sale.csv', encoding='gbk')
sale.head()

- SQL2数据过滤与排序
- 选择表中指定列

In [ ]:
import sqlite3 # sqlite3相当于轻量版，更多功能可使用SQLAlchemy

con = sqlite3.connect(':memory:') # 数据库连接
sale.to_sql('sale', con) # 将DataFrame注册成可用sql查询的表
newTable = pd.read_sql_query("select year, market, sale, profit from sale", con) # 也可使用read_sql
newTable.head()

- 选择表中所有列

In [ ]:
sqlResult = pd.read_sql_query('select * from sale', con)
sqlResult.head()

- 删除重复的行

In [ ]:
pd.read_sql_query("select DISTINCT  year from sale", con)

- 选择满足条件的行

In [ ]:
pd.read_sql_query("select * from sale where year=2012 and market='东'", con)

- 对行进行排序

In [ ]:
sql = '''select year, market, sale, profit
      from sale
      order by year'''
pd.read_sql_query(sql, con)

## 4.2纵向连接表
sql操作

In [ ]:
one = pd.read_csv("One.csv")
one.to_sql('One', con, index=False)
one.T

In [ ]:
two = pd.read_csv("Two.csv")
two.to_sql('Two', con, index=False)
two.T

union 和 union all

In [ ]:
union = pd.read_sql('select * from one UNION select * from two', con)
union_all = pd.read_sql('select * from one UNION ALL select * from two', con)
union.T

In [ ]:
union_all.T

except 和 intersect

In [ ]:
exceptTable = pd.read_sql('select * from one EXCEPT select * from two', con)
intersectTable = pd.read_sql('select * from one INTERSECT select * from two', con)
exceptTable.T

In [ ]:
intersectTable.T

*练习： 多表纵向连接

DataFrame操作

In [ ]:
pd.concat([one, two], axis=0, join='outer', ignore_index=True) # 更多参数可查看文档或帮助

## 4.3 横向连接表
sql操作

In [ ]:
table1 = pd.read_csv('Table1.csv')
table1.to_sql('table1', con, index=False)
table1.head()

In [ ]:
table2 = pd.read_csv('Table2.csv')
table2.to_sql('table2', con, index=False)
table2.head()

笛卡尔积

In [ ]:
pd.read_sql("select * from table1, table2", con)

内连接（使用inner join或使用where子句）

In [ ]:
pd.read_sql("select * from table1 as a inner join table2 as b on a.id=b.id", con)
# pd.read_sql("select * from table1 as a, table2 as b where a.id=b.id", con)

左连接

In [ ]:
pd.read_sql("select * from table1 as a left join table2 as b on a.id=b.id", con)

In [ ]:
# Attention: RIGHT and FULL OUTER JOINs are NOT currently supported, like:
# "select * from table1 as a RIGHT JOIN table2 as b on a.id=b.id"
# "select * from table1 as a FULL JOIN table2 as b on a.id=b.id"

DataFrame操作

In [ ]:
pd.merge(table1, table2, on='id', how='left') # 参数设置可查看帮助

按索引连接

In [ ]:
table1.join(table2, how='outer', lsuffix='t1', rsuffix='t2') # 参数设置可查看帮助

## 4.4 数据清洗

发现数据问题类型

In [ ]:
camp = pd.read_csv('teleco_camp_orig.csv')
camp.head()

In [ ]:
#pd.set_option('display.max_columns', None) # 设置显示所有列
# Try this: pd.describe_option('display')

- 脏数据或数据不正确

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(camp['AvgIncome'], bins=20, normed=True)
# Try this: accepts['purch_price'].plot(kind='hist')
# And this: sns.distplot(accepts['purch_price'], kde=True, fit=stats.norm) #should import seaborn as sns and scipy.stats first

In [ ]:
#这里的0值应该是缺失值
camp['AvgIncome']=camp['AvgIncome'].replace({0: np.NaN})
#像这种外部获取的数据要比较小心，经常出现意义不清晰或这错误值。AvgHomeValue也有这种情况
camp['AvgHomeValue']=camp['AvgHomeValue'].replace({0: np.NaN})
camp['Age']=camp['Age'].replace({0: np.NaN})
camp.head(8)

In [ ]:
camp['AvgIncome'].describe(include='all')

- 数据不一致-
这个问题需要详细的结合描述统计进行变量说明核对

- 数据重复

In [ ]:
camp['dup'] = camp.duplicated() # 生成重复标识变量
camp.dup.head()

In [ ]:
#本数据没有重复记录，此处只是示例
camp_dup = camp[camp['dup'] == True] # 把有重复的数据保存出来，以备核查
camp_nodup = camp[camp['dup'] == False] # 注意与camp.drop_duplicates()的区别
camp_nodup.head()

In [ ]:
camp['dup1'] = camp['ID'].duplicated() # 按照主键进行重复记录标识
# accepts['fico_score'].duplicated() # 没有实际意义

* 缺失值处理

In [ ]:
camp.describe()
#如果count数量少于样本量，说明存在缺失
#缺失最多的两个变量是Age和AvgIncome,缺失了大概20%。

In [ ]:
vmean = camp['Age'].mean(axis=0, skipna=True)
camp['Age_empflag'] = camp['Age'].isnull()
camp['Age']= camp['Age'].fillna(vmean)
camp['Age'].describe()

In [ ]:
vmean = camp['AvgHomeValue'].mean(axis=0, skipna=True)
camp['AvgHomeValue_empflag'] = camp['AvgHomeValue'].isnull()
camp['AvgHomeValue']= camp['AvgHomeValue'].fillna(vmean)
camp['AvgHomeValue'].describe()

In [ ]:
vmean = camp['AvgIncome'].mean(axis=0, skipna=True)
camp['AvgIncome_empflag'] = camp['AvgIncome'].isnull()
camp['AvgIncome']= camp['AvgIncome'].fillna(vmean)
camp['AvgIncome'].describe()

- 其他有缺失变量请自行填补，找到一个有缺失的分类变量，使用众数进行填补
- 多重插补：sklearn.preprocessing.Imputer仅可用于填补均值、中位数、众数，多重插补可考虑使用Orange、impute、Theano等包
- 多重插补的处理有两个要点：1、被解释变量有缺失值的观测不能填补，只能删除；2、只对放入模型的解释变量进行插补。

* 噪声值处理
- 盖帽法

In [ ]:
def blk(floor, root): # 'blk' will return a function
    def f(x):       
        if x < floor:
            x = floor
        elif x > root:
            x = root
        return x
    return f

q1 = camp['Age'].quantile(0.01) # 计算百分位数
q99 = camp['Age'].quantile(0.99)
blk_tot = blk(floor=q1, root=q99) # 'blk_tot' is a function
camp['Age']= camp['Age'].map(blk_tot)
camp['Age'].describe()

- 分箱（等深，等宽）
- 分箱法——等宽分箱

In [ ]:
 camp['Age_group1'] = pd.qcut( camp['Age'], 4) # 这里以age_oldest_tr字段等宽分为4段
camp.Age_group1.head()

- 分箱法——等深分箱

In [ ]:
 camp['Age_group2'] = pd.cut( camp['Age'], 4) # 这里以age_oldest_tr字段等宽分为4段
camp.Age_group2.head()

In [ ]:
camp.to_csv('tele_camp_ok.csv')